In [ ]:
#Libraries
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [ ]:
#Input URL link and define headers for requests
header={"content-type":"text"}
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64;     x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate",     "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
url="https://www.amazon.in/s?k=samsung+smartphones"
page_count=2
url_list=[url]

In [ ]:
#create list of urls for each page
list1=[''.join([url,"&page=",str(i)]) for i in range(2,page_count+1)]
url_list.extend(list1)
url_list

In [ ]:
#function to get soup of url
def get_soup(links):
    global headers
    page=req.get(links,headers=headers)
    status_code=page.status_code
    soup=bs(page.text,'lxml')
    return status_code,soup

In [ ]:
#Initializing lists to store scrapped data
product_ids=[]
product_name=[]
prices=[]
ratings=[]
customer_reviews=[]

In [ ]:
#function to extract page data
def extract_page_data(soup):
    global product_name
    global prices
    global ratings
    global customer_reviews
    for item in soup.select("div[class='sg-col-4-of-12 sg-col-8-of-16 sg-col-16-of-24 sg-col-12-of-20 sg-col-24-of-32 sg-col sg-col-28-of-36 sg-col-20-of-28']"):
        #extract product_name
        product_name.append(item.select_one("h2 span").get_text())
        #extract prices
        try:
            prices.append(item.find('span',class_="a-price-whole").get_text())
        except:
            prices.append("NaN")
        #extract ratings
        try:
            ratings.append(item.find('span',class_="a-icon-alt").get_text())
        except:
            ratings.append("NaN")
        try:
            customer_reviews.append(item.select_one('div[class="a-row a-size-small"] span[class="a-size-base"]').get_text())
        except:
            customer_reviews.append("NaN")

In [ ]:
#extract all page data
for page_url in url_list:
    get_soup
    stats_code,soup=get_soup(page_url)
    if stats_code!=200:
        print("Url request blocked")
        break
    else:
        extract_page_data(soup)

In [ ]:
#extracted data to dataframe
df=pd.DataFrame({'Prod_Name':product_name,'Price':prices,'Ratings':ratings,'No.of Cust Reviews':customer_reviews})
df.head()